In [1]:
import pandas as pd
import requests
import json

In [2]:
raw_df = pd.read_csv('flux-mobilite-domicile-lieu-travail-2018.csv').drop('Unnamed: 5', axis=1)
raw_df.head(5)

,Code residence,Name Residence,Code work,Name work,Flux
0,1004,Ambérieu-en-Bugey,69383,Lyon 3e Arrondissement,191
1,1043,Beynost,69266,Villeurbanne,138
2,1043,Beynost,69286,Rillieux-la-Pape,103
3,1043,Beynost,69383,Lyon 3e Arrondissement,133
4,1049,La Boisse,69383,Lyon 3e Arrondissement,121


In [3]:
coordinates = {}

for name in raw_df['Name Residence'].unique():
    name_clean = name
    name = name.replace(' ', '%20')
    url = f'https://api.opencagedata.com/geocode/v1/json?q={name}&key=03c48dae07364cabb7f121d8c1519492&no_annotations=1&language=en'

    response = requests.get(url, headers={})
    coordinates[name_clean] = json.loads(response.text)['results'][0]['geometry']

In [4]:
list_codes = [69381, 69382, 69383, 69384, 69385, 69386, 69387, 69388, 69389, 69029, 69152, 69244, 69142, 69034, 69149, 69081, 69089, 69202, 69266, 69259, 69204, 69256, 69290, 69199]

In [5]:
filtered_df = raw_df.loc[(raw_df['Code residence'].apply(lambda coord: coord in list_codes)) | (raw_df['Code work'].apply(lambda coord: coord in list_codes))]
filtered_df.to_clipboard(excel=True)

In [6]:
filtered_df2 = raw_df.loc[(raw_df['Code work'].apply(lambda coord: coord in list_codes))]
filtered_df2

,Code residence,Name Residence,Code work,Name work,Flux
0,1004,Ambérieu-en-Bugey,69383,Lyon 3e Arrondissement,191
1,1043,Beynost,69266,Villeurbanne,138
3,1043,Beynost,69383,Lyon 3e Arrondissement,133
4,1049,La Boisse,69383,Lyon 3e Arrondissement,121
5,1142,Dagneux,69383,Lyon 3e Arrondissement,147
...,...,...,...,...,...
1042,69389,Lyon 9e Arrondissement,69384,Lyon 4e Arrondissement,488
1043,69389,Lyon 9e Arrondissement,69385,Lyon 5e Arrondissement,538
1044,69389,Lyon 9e Arrondissement,69386,Lyon 6e Arrondissement,647
1045,69389,Lyon 9e Arrondissement,69387,Lyon 7e Arrondissement,855


In [7]:
filtered_allin = filtered_df2.loc[(filtered_df2['Code residence'].apply(lambda coord: coord in list_codes))]

In [8]:
temp_pivot = filtered_allin.groupby('Code residence')[['Flux']].sum()
temp_pivot = temp_pivot.reset_index().rename(columns={'Code residence': 'Code', 'Flux': 'Flux out'}).set_index('Code')

temp_pivot2 = filtered_allin.groupby('Code work')[['Flux']].sum()
temp_pivot2 = temp_pivot2.reset_index().rename(columns={'Code work': 'Code', 'Flux': 'Flux in'}).set_index('Code')

pivot_all_in = temp_pivot.join(temp_pivot2)

In [9]:
codes_ordered = raw_df['Code residence'].unique()
code_coordinates = {codes_ordered[i]: coordinates[list(coordinates.keys())[i]] for i in range(len(codes_ordered))}

pivot_all_in = pivot_all_in.reset_index()
# replace '.' by ',' bc pt
pivot_all_in['lat'] = pivot_all_in['Code'].apply(lambda code: str(code_coordinates[code]['lat']).replace('.', ','))
pivot_all_in['lng'] = pivot_all_in['Code'].apply(lambda code: str(code_coordinates[code]['lng']).replace('.', ','))
pivot_all_in.to_clipboard(excel=True)

In [10]:
filtered_allin['lat residence'] = filtered_allin['Code residence'].apply(lambda code: code_coordinates[code]['lat'])
filtered_allin['lgn residence'] = filtered_allin['Code residence'].apply(lambda code: code_coordinates[code]['lng'])

filtered_allin['lat work'] = filtered_allin['Code work'].apply(lambda code: code_coordinates[code]['lat'])
filtered_allin['lgn work'] = filtered_allin['Code work'].apply(lambda code: code_coordinates[code]['lng'])


C:\Users\tulio\AppData\Local\Temp/ipykernel_108/1753851308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_allin['lat residence'] = filtered_allin['Code residence'].apply(lambda code: code_coordinates[code]['lat'])
C:\Users\tulio\AppData\Local\Temp/ipykernel_108/1753851308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_allin['lgn residence'] = filtered_allin['Code residence'].apply(lambda code: code_coordinates[code]['lng'])
C:\Users\tulio\AppData\Local\Temp/ipykernel_108/17538513

In [11]:
filtered_allin.to_csv('filtered_in.csv')

In [12]:
pivot_all_in


,Code,Flux out,Flux in,lat,lng
0,69029,8418,10465.0,"45,7337532","4,9092352"
1,69034,10227,3668.0,"45,7969952","4,8423304"
2,69081,2367,4486.0,"45,7754498","4,7785644"
3,69089,2949,515.0,"47,4542392","4,8817511"
4,69142,693,NaN,"45,7292173","4,8096049"
5,69149,6412,2201.0,"45,7153254","4,8039331"
6,69152,1414,3017.0,"45,7028321","4,8241323"
7,69199,3121,1783.0,"45,7001","4,84935"
8,69202,4724,969.0,"45,7362255","4,7934767"
9,69204,3690,2083.0,"45,6950021","4,7928827"


ModuleNotFoundError: No module named 'geopandas'